In [7]:
import descartes, matplotlib.pyplot as plt 
import pandas as pd
import censusdata
import string

In [2]:
# NOTE: determined fps data has misaligned rows.  Working to fix!
#fpsgeo = ["../AssessingParcels"+str(i) +"/ASSESSING_ParcelsFY"+str(i)+".shp" for i in range(2016, 2021)]

fpsdata = ["../AssessingParcels"+str(i) +"/ASSESSING_PropertyDatabase_FY"+str(i)+".csv" for i in range(2016, 2021)]
#parcels = [gpd.read_file(fpsgeo[i]) for i in range(5)]
dtypes = {"PID": 'int32', "GISID": 'object', 'BldgNum': 'Int8', 'Address': 'object',\
                                               "Unit": 'object', 'StateClassCode': 'Int16', 'PropertyClass': 'object', \
                                               'Zoning': 'object', 'MapLot': 'object', 'LandArea': 'object', 'Year': 'Int16',\
                                               'TaxDistrict': 'object', 'ResidentialExemption': 'object', 'Building': 'Int32',\
                                               'LandValue': 'Int32', 'AssessedValue': 'Int32', 'SalePrice': 'Int32',\
                                               'BookPage': 'object', 'SaleDate': 'datetime64', 'PreviousAssessedValue':'Int32',\
                                               'PropertyTaxAmount':'Int32','Owner_Name': 'object', 'Owner_CoOwnerName': 'object',\
                                               'Owner_Address': 'object', 'Owner_Address2': 'object', 'Owner_City': 'object',\
                                               'Owner_State': 'object', 'Owner_Zip':'object', 'Exterior_Style': 'object',\
                                               'Exterior_Occupancy': 'object', 'Exterior_NumStories': 'float16', \
                                               'Exterior_WallType': 'object', 'Exterior_WallHeight': 'float16', \
                                               'Exterior_RoofType': 'object', 'Exterior_RoofMaterial': 'object', \
                                               'Exterior_FloorLocation': 'Int8', 'Exterior_View': 'object', \
                                               'Interior_LivingArea': 'Int32', 'Interior_NumUnits': 'Int8', \
                                               'Interior_TotalRooms': 'Int8', 'Interior_Bedrooms': 'Int8', \
                                               'Interior_Kitchens': 'object', 'Interior_FullBaths': 'Int8', 'Interior_HalfBaths': 'Int8',\
                                               'Interior_Fireplaces': 'Int8', 'Interior_Flooring': 'object', 'Interior_Layout': 'object',\
                                               'Interior_LaundryInUnit':'Int8', 'Systems_HeatType': 'object', 'Systems_HeatFuel': 'object',\
                                               'Systems_CentralAir': 'Int8', 'Systems_Plumbing': 'object', 'Condition_YearBuilt': 'Int16',\
                                               'Condition_InteriorCondition': 'object', 'Condition_OverallCondition': 'object',\
                                               'Condition_OverallGrade': 'object', 'Parking_Open': 'Int8', 'Parking_Covered':'Int8',\
                                               'Parking_Garage': 'Int8', 'UnfinishedBasementGross':'Int16', \
                                                'FinishedBasementGross':'Int16'}

In [ ]:
fpsdata = ["../AssessingParcels"+str(i) +"/ASSESSING_PropertyDatabase_FY"+str(i)+".csv" for i in range(2016, 2021)]
x = pd.read_csv(fpsdata[0], low_memory=False)

In [ ]:
x.columns

In [ ]:
x['asv'] = x['SalePrice']/x['AssessedValue']

In [ ]:
to_look_at = x[x["PropertyClass"]=="SNGL-FAM-RES"]

In [ ]:
not_null = to_look_at[to_look_at["SalePrice"].notnull()][to_look_at["AssessedValue"].notnull()]

In [ ]:
not_null["asv"] = not_null["SalePrice"]/not_null["AssessedValue"]

In [ ]:
filter_out_low_values = not_null[not_null["asv"]>0.05]

In [ ]:
filter_out_low_values["asv"].describe()

In [59]:
parcel_data = [pd.read_csv(fpsdata[i]) for i in range(0, 5)]
firstyear_parcels = parcel_data[4]
sale_data = pd.read_csv("../RawSales/LA3ParcelSearch.csv")
sale_data['Parcel Id'] = sale_data['Parcel Id'].str.replace('_','-')
sale_data['Parcel Id'] = sale_data['Parcel Id'].str.replace('--','-')
sale_data['Parcel Id'] = sale_data['Parcel Id'].str.replace('--','-')
sale_data['Parcel Id'] = sale_data['Parcel Id'].str.strip(string.punctuation)
merged_sales_assessments = sale_data.merge(firstyear_parcels, left_on='Parcel Id', right_on='MapLot', how="outer", indicator = True)

In [60]:
merged_sales_assessments['_merge'].value_counts()

right_only    19769
both          14359
left_only       887
Name: _merge, dtype: int64

In [61]:
merged_sales_assessments[merged_sales_assessments['_merge']=="left_only"].head

<bound method NDFrame.head of       City/ Town  Fiscal Year Process   Sale Date      Parcel Id  \
51     Cambridge       2005.0       C  01/15/2003      122-166-2   
296    Cambridge       2005.0       C  04/14/2003      11-43-PH3   
297    Cambridge       2008.0       C  09/29/2006      11-43-PH3   
298    Cambridge       2008.0       E  09/29/2006      11-43-PH3   
1036   Cambridge       2005.0       C  08/07/2003         271-47   
...          ...          ...     ...         ...            ...   
13654  Cambridge       2017.0       C  10/06/2015         24-130   
13807  Cambridge       2018.0       I  03/04/2016        19-28-1   
13895  Cambridge       2018.0       I  05/17/2016       242A-132   
14279  Cambridge       2018.0       I  12/09/2016        127-171   
14972  Cambridge       2020.0       I  07/12/2018  101-147-411/1   

                                         Seller  \
51                            LIVE, THEODORE R.   
296                     PASCAVAGE, MICHAEL H. &   
